<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/static/images/logo-v4.svg" width=200/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/annotation_import/basics.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/annotation_import/basics.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Annotation Imports
* This notebook is a high level introduction demonstrating multiple ways to upload your annotations. This will cover the following:
    * Model-assisted labeling - used to provide pre-annotated data for your labelers. This will enable a reduction in the total amount of time to properly label your assets. Model-assisted labeling does not submit the labels automatically, and will need to be reviewed by a labeler for submission.
    * Label Import - used to provide ground truth labels. These can in turn be used and compared against prediction labels, or used as benchmarks to see how your labelers are doing.


* For complete examples see image_mal.ipynb or ner_mal.ipynb

* For information on what types of annotations are supported per data type, refer to this [documentation](https://docs.labelbox.com/docs/model-assisted-labeling#option-1-import-via-python-annotation-types-recommended)

* Notes:
    * If you are importing more than 1,000 mask annotations at a time, consider submitting separate jobs, as they can take longer than other annotation types to import.
    * Wait until the import job is complete before opening the Editor to make sure all annotations are imported properly.

# Installs

In [ ]:
!pip install -q 'labelbox[data]'

# Imports

In [ ]:
import labelbox
from labelbox.schema.ontology import OntologyBuilder, Tool
from labelbox import Client, LabelingFrontend, LabelImport, MALPredictionImport, MediaType
from labelbox.schema.queue_mode import QueueMode
from labelbox.data.annotation_types import (
    Label, ImageData, ObjectAnnotation, Rectangle, Point, LabelList
)
from labelbox.data.serialization import NDJsonConverter
import uuid
print(labelbox.__version__)

3.27.1


# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your api key
API_KEY = None
client = Client(api_key=API_KEY)

---- 
### Steps
1. Make sure project is setup
2. Collect annotations
3. Upload

### Project setup

We will be creating two projects, one for model-assisted labeling, and one for label imports

In [ ]:
# Only update this if you have an on-prem deployment
ontology_builder = OntologyBuilder(tools=[
    Tool(tool=Tool.Type.BBOX, name="box")
])


mal_project = client.create_project(name="image_mal_project",
                                    queue_mode=QueueMode.Batch,
                                    media_type=MediaType.Image)
li_project = client.create_project(name="image_label_import_project",
                                    queue_mode=QueueMode.Batch,
                                    media_type=MediaType.Image)


dataset = client.create_dataset(name="annotation_import_demo_dataset")

test_img_url = {
    "row_data": "https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg",
    "global_key": "TEST-ID %id" % uuid.uuid1()
}

data_row = dataset.create_data_row(test_img_url)
print(data_row)

<DataRow {'created_at': datetime.datetime(2022, 9, 22, 18, 58, 1, tzinfo=datetime.timezone.utc), 'external_id': None, 'global_key': 'TEST-ID 163726426813019675679242181306463551490d', 'media_attributes': {}, 'metadata': [], 'metadata_fields': [], 'row_data': 'https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg', 'uid': 'cl8df2mda0bi00700hrxghneh', 'updated_at': datetime.datetime(2022, 9, 22, 18, 58, 1, tzinfo=datetime.timezone.utc)}>


In [ ]:
# Setup Batches and Ontology

# We need the data row ID to create a batch
batch_datarows = [dr.uid for dr in list(dataset.export_data_rows())]

# Create a batch to send to your MAL project
batch_mal = mal_project.create_batch(
  "first-batch-MAL", # Each batch in a project must have a unique name
  batch_datarows, # A list of data rows or data row ids
  5 # priority between 1(Highest) - 5(lowest)
)

# Create a batch to send to you LI project
batch_li = li_project.create_batch(
    "first-batch-LI", # Each batch in a project must have a unique name
    batch_datarows, # A list of data rows or data row ids
    5 # priority between 1(Highest) - 5(lowest)
)

# Setup your ontology / labeling editor
editor = next(client.get_labeling_frontends(where=LabelingFrontend.name == "Editor")) # Unless using a custom editor,
# Connect your ontology and editor to your MAL and LI project
mal_project.setup(editor, ontology_builder.asdict())
li_project.setup(editor, ontology_builder.asdict())

print("Batch Li: ", batch_li)
print("Batch Mal: ", batch_mal)

Batch Li:  <Batch {'created_at': datetime.datetime(2022, 9, 22, 18, 58, 53, tzinfo=datetime.timezone.utc), 'name': 'first-batch-LI', 'size': 1, 'uid': '9a6a06f0-3aa8-11ed-b5f9-814288043136', 'updated_at': datetime.datetime(2022, 9, 22, 18, 58, 53, tzinfo=datetime.timezone.utc)}>
Batch Mal:  <Batch {'created_at': datetime.datetime(2022, 9, 22, 18, 58, 52, tzinfo=datetime.timezone.utc), 'name': 'first-batch-MAL', 'size': 1, 'uid': '995eb170-3aa8-11ed-a4fe-f17c753c66ec', 'updated_at': datetime.datetime(2022, 9, 22, 18, 58, 52, tzinfo=datetime.timezone.utc)}>


#### Create Label using Annotation Type Objects
* It is recommended to use the Python SDK's annotation types. Below is an example of a bounding box, which is the Rectangle annotation type

* A more in depth example can be found [here](https://docs.labelbox.com/docs/bounding-box-json)

In [ ]:
# Create a label with the related data and annotations
rectangle = Rectangle(start=Point(x=30,y=30), end=Point(x=200,y=200))
rectangle_annotation = ObjectAnnotation(value=rectangle, name="box")
image_data = ImageData(uid=data_row.uid)

label = Label(
    data=image_data,
    annotations = [rectangle_annotation]
)

label

Label(uid=None, data=ImageData(im_bytes=None,file_path=None,url=None,arr=None), annotations=[ObjectAnnotation(name='box', feature_schema_id=None, extra={}, value=Rectangle(extra={}, start=Point(extra={}, x=30.0, y=30.0), end=Point(extra={}, x=200.0, y=200.0)), classifications=[])], extra={})

### Model Assisted Labeling 

To do model-assisted labeling, we need to convert a Label object into an NDJSON. 

This is easily done with using the NDJSONConverter class

We will create a Label called mal_label which has the same original structure as the label above

Notes:
* the NDJsonConverter takes in a list of labels

In [ ]:
mal_label = Label(
    data=image_data,
    annotations = [rectangle_annotation]
)
mal_label_list = LabelList()
mal_label_list.append(mal_label)

mal_ndjson = list(NDJsonConverter.serialize(mal_label_list))

mal_ndjson

[{'uuid': 'b8ca0793-c928-4129-ac34-9116a5bfb01d',
  'dataRow': {'id': 'cl8df2mda0bi00700hrxghneh'},
  'name': 'box',
  'classifications': [],
  'bbox': {'top': 30.0, 'left': 30.0, 'height': 170.0, 'width': 170.0}}]

In [ ]:
upload_job = MALPredictionImport.create_from_objects(
    client = client, 
    project_id = mal_project.uid, 
    name="upload_label_import_job", 
    predictions=mal_ndjson)

In [ ]:
# Errors will appear for each annotation that failed.
# This will provide information only after the upload_job is complete, so we do not need to worry about having to rerun
upload_job.wait_until_done()
print("Errors:", upload_job.errors)

Errors: []


### Label Import

Label import is very similar to model-assisted labeling. We will create a Label called li_label which has the same original structure as the label above

In [ ]:
li_label = Label(
    data=image_data,
    annotations = [rectangle_annotation]
)
li_label_list = LabelList()
li_label_list.append(li_label)
ndjson_labels = list(NDJsonConverter.serialize(li_label_list))

ndjson_labels

[{'uuid': 'aa10eafa-6f0a-41d5-8c24-6a8bb9fae189',
  'dataRow': {'id': 'cl8df2mda0bi00700hrxghneh'},
  'name': 'box',
  'classifications': [],
  'bbox': {'top': 30.0, 'left': 30.0, 'height': 170.0, 'width': 170.0}}]

In [ ]:
upload_job = LabelImport.create_from_objects(
    client = client, 
    project_id = li_project.uid, 
    name="upload_label_import_job", 
    labels=ndjson_labels)

In [ ]:
upload_job.wait_until_done()
print("Errors:", upload_job.errors)


Errors: []
